<a href="https://colab.research.google.com/github/Yoppy-0808/Earthquake_area_pred/blob/main/LightGBM%E3%81%AB%E3%82%88%E3%82%8B%E5%9C%B0%E9%9C%87%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CSVファイルの取得

In [1]:
# 1919~2025までのCSVファイル作成のためのコード
import time
import requests # Requests：分かりやすい形式でWebページやWeb APIにアクセスし、結果を取得
import pandas as pd
from datetime import datetime, timedelta

API_URL = "https://www.data.jma.go.jp/eqdb/data/shindo/api/"
OUT_PATH = "jma_shindo_19192025.csv"#保存先

#取得データ内容指示
BASE_PAYLOAD = {
    "mode": "search",
    "mag[]": ["0.0", "9.9"],
    "dep[]": ["000", "999"],
    "epi[]": ["99"],
    "pref[]": ["99"],
    "city[]": ["99"],
    "station[]": ["99"],
    "obsInt": "1",
    "maxInt": "2",
    "additionalC": "false",
    "Sort": "S0",
    "Comp": "C0",
    "seisCount": "false",
    "observed": "false",
}

TIMEOUT = 30  #取得時間の設定をし、反応がなかったら失敗にする
SLEEP_SEC = 0.2           # アクセス過多を避ける
MAX_RETRY = 5 # 失敗したら最大5回までやり直す
MIN_GRANULARITY = timedelta(minutes=1)  # 最小の時間幅として、1分単位にしている

def _post_with_retry(payload: dict) -> dict:
    last_err = None #エラーの記憶箇所（exceptの場合）
    for i in range(MAX_RETRY):
        try:
            r = requests.post(API_URL, data=payload, timeout=TIMEOUT) #取得条件
            r.raise_for_status()#HTTPが 200番台じゃなければエラーを発生させる
            return r.json() #内容を JSON として Pythonの辞書にして返す
        except Exception as e:
            last_err = e
            time.sleep(1.0 * (i + 1)) #サーバーが一時的に混んでる時用
    raise RuntimeError(f"API request failed after retries: {last_err}")

def fetch_range(dt_from: datetime, dt_to: datetime) -> list[dict]:

# [dt_from, dt_to] を1回で取得して res の list を返す

    payload = dict(BASE_PAYLOAD) # 元変数を書き換えないように
    # 年数
    payload["dateTimeF[]"] = [dt_from.strftime("%Y-%m-%d"), dt_from.strftime("%H:%M")]
    # 時間
    payload["dateTimeT[]"] = [dt_to.strftime("%Y-%m-%d"), dt_to.strftime("%H:%M")]

    data = _post_with_retry(payload)
    rows = data.get("res", []) or [] #変な返り方をしても、必ずリストにするため
    return rows

# 1000件制限を回避するため、区間を分割して全件取得する
def fetch_all(dt_from: datetime, dt_to: datetime) -> pd.DataFrame:

    stack = [(dt_from, dt_to)]
    collected = [] #集めた地震データを全部入れるリスト
    seen_keys = set() #同じデータを二重に保存しない

    while stack:
        a, b = stack.pop()###

        rows = fetch_range(a, b)###
        time.sleep(SLEEP_SEC)

        # 1000件に達しているなら、まだある可能性が高いので分割
        if len(rows) >= 1000:
            if (b - a) <= MIN_GRANULARITY:
                # 1分まで分割しても1000件以上＝このAPI仕様の範囲では取りきれない
                raise RuntimeError(
                    f"Too many records even in minimal granularity range: {a} - {b} (rows={len(rows)})"
                )

            mid = a + (b - a) / 2 #期間の真ん中の日時を作る
            # 端の取りこぼし防止のため、後半は mid+1分 ではなくそのまま mid を境界に分割

            #前半・後半の2つの期間をスタックに積む
            stack.append((a, mid))
            stack.append((mid, b))
            continue

        # 1000件未満なら確定として回収
        for r in rows: #r は1件分の地震データ（辞書)
            # 重複排除キー（APIの戻りにユニークIDがあればそれを優先）
            # 汎用的に、発生時刻＋震央＋Mなどを連結してキー化（不足なら列を増やす）
            key = (
                str(r.get("ot", "")),
                str(r.get("eid", "")),
                str(r.get("mag", "")),
                str(r.get("dep", "")),
                str(r.get("lat", "")),
                str(r.get("lon", "")),
            )
            if key in seen_keys:
                continue
            seen_keys.add(key)
            collected.append(r)

        print(f"OK {a} - {b}  rows={len(rows)}  total={len(collected)}")

# DataFrameでもできるが、辞書が入れ子の場合に備えてjson_normalizeを利用する
    df = pd.json_normalize(collected)

#ot_dt があればそれで並べ替え、なければ ot で並べ替える
    if "ot" in df.columns:
        df["ot_dt"] = pd.to_datetime(df["ot"], errors="coerce") #変換できないものは NaTに

    # 安全のため時系列で並べる
    sort_cols = [c for c in ["ot_dt", "ot"] if c in df.columns]
    if sort_cols:
        df = df.sort_values(sort_cols).reset_index(drop=True)

    return df

if __name__ == "__main__":
    start = datetime(1919, 1, 1, 0, 0)
    end   = datetime(2025, 12, 31, 23, 59)

    df_all = fetch_all(start, end)

#encoding="utf-8-sig" はExcelで文字化けしにくい保存形式
    df_all.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
    print(f"saved: {OUT_PATH} rows={len(df_all)} cols={len(df_all.columns)}")

OK 2025-08-01 08:02:45.234375 - 2025-12-31 23:59:00  rows=404  total=404
OK 2025-05-17 00:04:37.851562 - 2025-08-01 08:02:45.234375  rows=878  total=1282
OK 2025-03-01 16:06:30.468750 - 2025-05-17 00:04:37.851562  rows=134  total=1416
OK 2024-04-30 08:14:00.937500 - 2025-03-01 16:06:30.468750  rows=546  total=1962
OK 2023-11-29 16:17:46.171875 - 2024-04-30 08:14:00.937500  rows=957  total=2919
OK 2023-06-30 00:21:31.406250 - 2023-11-29 16:17:46.171875  rows=322  total=3241
OK 2022-08-28 16:29:01.875000 - 2023-06-30 00:21:31.406250  rows=583  total=3824
OK 2021-10-27 08:36:32.343750 - 2022-08-28 16:29:01.875000  rows=711  total=4535
OK 2020-12-26 00:44:02.812500 - 2021-10-27 08:36:32.343750  rows=628  total=5163
OK 2019-04-25 08:59:03.750000 - 2020-12-26 00:44:02.812500  rows=938  total=6101
OK 2018-06-24 01:06:34.218750 - 2019-04-25 08:59:03.750000  rows=642  total=6743
OK 2017-08-22 17:14:04.687500 - 2018-06-24 01:06:34.218750  rows=572  total=7315
OK 2016-10-21 09:21:35.156250 - 2017

### 前処理

In [2]:
import pandas as pd
import numpy as np
import re

# df作成（元データの取り込み）
df = pd.read_csv("jma_shindo_19192025.csv")

# 文字列正規化: 全角数字⇒半角（震度部分）
_ZEN2HAN = str.maketrans("０１２３４５６７８９．－", "0123456789.-")

# ot を datetime にする（失敗は NaT になる） ---
df["ot"] = pd.to_datetime(df["ot"], errors="coerce")
df["ot_year"]  = df["ot"].dt.year.astype("Float64")
df["ot_month"] = df["ot"].dt.month.astype("Float64")
df["ot_day"]   = df["ot"].dt.day.astype("Float64")
df["ot_hour"]  = df["ot"].dt.hour.astype("Float64")
# ot_dt は削除 ---
df = df.drop(columns=["ot_dt"], errors="ignore")

# dep のダミー化（数値列 dep_num を作る）
def dep_to_km(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    # "15 km" / "0 km" を想定
    m = re.search(r"(\d+(\.\d+)?)\s*km", s, flags=re.IGNORECASE)
    if m:
        return float(m.group(1))
    # "不明" 等は NaN
    return np.nan

# maxI のダミー化（数値列 maxI_num を作る）
def normalize_shindo(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    if s == "" or "不明" in s:
        return None

    # 全角→半角、空白除去
    s = s.translate(_ZEN2HAN)
    s = re.sub(r"\s+", "", s)

    # 例: "震度5強", "5強", "震度５強" などの揺れを吸収
    s = s.replace("震度", "")
    return s

    # 震度を数値化:5弱->5.0, 5強->5.5, 6弱->6.0, 6強->6.5, 7->7.0
    # 0〜4 はそのまま float
def shindo_to_num(x):
    s = normalize_shindo(x)
    if s is None:
        return np.nan

    # 代表的な強弱
    if s in ("5弱", "5-"):   # もし "5-" のような表記が混じる場合も吸収
        return 5.0
    if s == "5強" or s == "5+":
        return 5.5
    if s in ("6弱", "6-"):
        return 6.0
    if s == "6強" or s == "6+":
        return 6.5
    if s == "7":
        return 7.0

    # 0〜4, または "5" "6" のように強弱無し表記
    m = re.fullmatch(r"([0-7](?:\.\d+)?)", s)
    if m:
        return float(m.group(1))
    # 想定外表記は NaN
    return np.nan

df["dep_num"] = df["dep"].apply(dep_to_km)
# maxI_num を作り直し
df["maxI_num"] = df["maxI"].apply(shindo_to_num).astype("Float64")
# 保存
df.to_csv("jma_shindo_19192025dum.csv", index=False)

In [6]:
import pandas as pd
import json
import numpy as np
from scipy.spatial import cKDTree

DUM_CSV      = "jma_shindo_19192025dum.csv"
STATIONS_JSON= "stations.json"
OUT_REFINED  = "jma_shindo_refined.csv"

# =========================
# 1) 読み込み
# =========================
df = pd.read_csv(DUM_CSV)

with open(STATIONS_JSON, "r", encoding="utf-8") as f:
    stations_data = json.load(f)

# =========================
# 2) lat/lon を数値化（壊れた値は NaN）
# =========================
df["lat"] = pd.to_numeric(df.get("lat"), errors="coerce")
df["lon"] = pd.to_numeric(df.get("lon"), errors="coerce")

# =========================
# 3) ot をdatetime化 → ot_year/month/day/hour を再生成
# =========================
df["ot"] = pd.to_datetime(df.get("ot"), errors="coerce")
df = df.dropna(subset=["ot"]).copy()

df["ot_year"]  = df["ot"].dt.year
df["ot_month"] = df["ot"].dt.month
df["ot_day"]   = df["ot"].dt.day
df["ot_hour"]  = df["ot"].dt.hour

# =========================
# 4) 数値化
#    - mag：数値化するが補完しない
#    - maxI_num：数値化のみ（欠損は震度分析では除外する前提）
#    - dep_num：数値化 + 中央値補完
# =========================
df["mag"] = pd.to_numeric(df.get("mag"), errors="coerce")         # 補完しない
df["maxI_num"] = pd.to_numeric(df.get("maxI_num"), errors="coerce")

df["dep_num"] = pd.to_numeric(df.get("dep_num"), errors="coerce")
dep_median = df["dep_num"].median()
df["dep_num"] = df["dep_num"].fillna(dep_median)

# 深さが負になる等の変なのがあれば0に丸め
df["dep_num"] = df["dep_num"].clip(lower=0)

# =========================
# 5) KDTreeで「最寄り観測点 → 都道府県」を付与
#    ※ lat/lon が NaN の行はKDTree検索できないので、最後に処理分け
# =========================
coords = []
prefs  = []
codes  = []
for s in stations_data:
    coords.append([float(s["lat"]), float(s["lon"])])
    prefs.append(s["pref"]["name"])
    codes.append(s["pref"]["code"])

tree = cKDTree(coords)

# 検索可能（lat/lon がある）な行だけ
mask_ok = df["lat"].notna() & df["lon"].notna()
quake_coords = df.loc[mask_ok, ["lat", "lon"]].values
distances, indices = tree.query(quake_coords)

df.loc[mask_ok, "edit_name"] = [prefs[i] for i in indices]
df.loc[mask_ok, "pref_code"] = [codes[i] for i in indices]

# lat/lon が無い行は不明扱い
df.loc[~mask_ok, "edit_name"] = np.nan
df.loc[~mask_ok, "pref_code"] = np.nan

# pref_code を int 化（欠損はそのまま）
df["pref_code"] = pd.to_numeric(df["pref_code"], errors="coerce")

df = df.dropna(subset=["pref_code"]).copy()
df["pref_code"] = df["pref_code"].astype(int)

# =========================
# 6) カラム整理（学習で使う前提の柱だけは確実に残す）
# =========================
keep_cols = [
    "id", "ot", "name",
    "lat", "lon",
    "dep_num", "mag", "maxI_num",
    "ot_year", "ot_month", "ot_day", "ot_hour",
    "edit_name", "pref_code"
]
# 存在するものだけ残す
keep_cols = [c for c in keep_cols if c in df.columns]
df = df[keep_cols].copy()

# =========================
# 7) 保存
# =========================
df.to_csv(OUT_REFINED, index=False, encoding="utf-8-sig")
print("saved:", OUT_REFINED)
print(df[["name","edit_name","pref_code"]].head(10))
print("rows:", len(df), "pref_nunique:", df["pref_code"].nunique())

saved: jma_shindo_refined.csv
      name edit_name  pref_code
0      十勝沖       北海道          1
1    千葉県南部       千葉県         12
2    長野県南部       長野県         20
3     鳥島近海       東京都         13
4   千葉県北西部       茨城県          8
5    埼玉県南部       埼玉県         11
6   千葉県北西部       千葉県         12
7     岩手県沖       岩手県          3
8  沖縄本島北西沖       沖縄県         47
9    大分県北部       大分県         44
rows: 53148 pref_nunique: 47


### 10日間モデルの作成

In [7]:
import numpy as np
import pandas as pd
import joblib

from lightgbm import LGBMClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import average_precision_score, brier_score_loss

REFINED_CSV = "jma_shindo_refined.csv"
# OUT_MODEL   = "eq_pref_10d_I4p_model_ver10.joblib"
# OUT_MODEL   = "eq_pref_10d_I4p_model_ver10_90rm.joblib"
OUT_MODEL   = "eq_pref_10d_I4p_model_ver10_90rm_train.joblib"

# =========================
# 1) refined 読み込み
# =========================
df = pd.read_csv(REFINED_CSV, parse_dates=["ot"])
df = df.dropna(subset=["ot"]).copy()

df["date"] = df["ot"].dt.floor("D")
df["pref_key"] = df["pref_code"].astype(int)

# 数値列の安全化（refinedで整っている想定だが念のため）
df["maxI_num"] = pd.to_numeric(df["maxI_num"], errors="coerce")
df["mag"] = pd.to_numeric(df["mag"], errors="coerce")          # 補完しない
df["dep_num"] = pd.to_numeric(df["dep_num"], errors="coerce")  # refinedで補完済みの想定

# =========================
# 2) 日次パネル作成（pref×date）
# =========================
daily_cnt = df.groupby(["pref_key","date"]).size().rename("cnt").reset_index()

df_i4 = df[df["maxI_num"].notna() & (df["maxI_num"] >= 4.0)]
daily_i4 = df_i4.groupby(["pref_key","date"]).size().rename("cnt_I4p").reset_index()

daily_mag = df.groupby(["pref_key","date"])["mag"].mean().rename("mag_mean").reset_index()
daily_dep = df.groupby(["pref_key","date"])["dep_num"].mean().rename("dep_mean").reset_index()

daily = (daily_cnt
         .merge(daily_i4, on=["pref_key","date"], how="left")
         .merge(daily_mag, on=["pref_key","date"], how="left")
         .merge(daily_dep, on=["pref_key","date"], how="left"))

daily["cnt_I4p"] = daily["cnt_I4p"].fillna(0).astype(int)
daily["mag_mean"] = daily["mag_mean"].fillna(0)   # ※magは補完しない方針だが、日次平均は0埋めして特徴量として扱う
daily["dep_mean"] = daily["dep_mean"].fillna(0)

# 全県×全日埋め（1..47固定）
all_dates = pd.date_range(daily["date"].min(), daily["date"].max(), freq="D")
prefs = np.arange(1, 48)
grid = pd.MultiIndex.from_product([prefs, all_dates], names=["pref_key","date"]).to_frame(index=False)

daily_full = grid.merge(daily, on=["pref_key","date"], how="left")
daily_full["cnt"] = daily_full["cnt"].fillna(0).astype(int)
daily_full["cnt_I4p"] = daily_full["cnt_I4p"].fillna(0).astype(int)
daily_full["mag_mean"] = daily_full["mag_mean"].fillna(0)
daily_full["dep_mean"] = daily_full["dep_mean"].fillna(0)
daily_full = daily_full.sort_values(["pref_key","date"]).reset_index(drop=True)

# =========================
# 3) 特徴量
# =========================
def add_features(d):
    d = d.copy()
    g = d.groupby("pref_key", group_keys=False)

    for w in [3,7,14]:
        d[f"cnt_{w}d"]      = g["cnt"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"cntI4_{w}d"]    = g["cnt_I4p"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"mag_{w}d_mean"] = g["mag_mean"].rolling(w, min_periods=1).mean().reset_index(level=0, drop=True)
        d[f"dep_{w}d_mean"] = g["dep_mean"].rolling(w, min_periods=1).mean().reset_index(level=0, drop=True)

    # for w in [10,30,90]:
    for w in [10,30]:
        d[f"cnt_{w}d"]   = g["cnt"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"cntI4_{w}d"] = g["cnt_I4p"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)

    # days_since_any_eq
    d["any_eq_today"] = (d["cnt"] > 0).astype(int)
    last_eq = np.where(d["any_eq_today"].to_numpy() > 0, d["date"].to_numpy(dtype="datetime64[D]"), np.datetime64("NaT"))
    d["last_eq_date"] = pd.Series(last_eq)
    d["last_eq_date"] = g["last_eq_date"].ffill()
    diff = (d["date"].to_numpy(dtype="datetime64[D]") - d["last_eq_date"].to_numpy(dtype="datetime64[D]")).astype("timedelta64[D]")
    d["days_since_any_eq"] = np.where(pd.isna(d["last_eq_date"]), 9999, diff.astype(int))

    # days_since_I4p
    d["i4_today"] = (d["cnt_I4p"] > 0).astype(int)
    last_i4 = np.where(d["i4_today"].to_numpy() > 0, d["date"].to_numpy(dtype="datetime64[D]"), np.datetime64("NaT"))
    d["last_i4_date"] = pd.Series(last_i4)
    d["last_i4_date"] = g["last_i4_date"].ffill()
    diff2 = (d["date"].to_numpy(dtype="datetime64[D]") - d["last_i4_date"].to_numpy(dtype="datetime64[D]")).astype("timedelta64[D]")
    d["days_since_I4p"] = np.where(pd.isna(d["last_i4_date"]), 9999, diff2.astype(int))

    d = d.drop(columns=["last_eq_date","last_i4_date"], errors="ignore")
    return d

feat = add_features(daily_full)

# =========================
# 4) 目的変数 y（次の10日でI4+が1回でもあれば1）
#    ※未来リーク防止：prefごとにshift
# =========================
H = 10
g = feat.groupby("pref_key", group_keys=False)
future_i4 = g["cnt_I4p"].rolling(H, min_periods=1).sum().reset_index(level=0, drop=True).shift(-H)
feat["y"] = (future_i4.fillna(0) > 0).astype(int)

# =========================
# 5) 学習/テスト（時系列ホールドアウト）
# =========================
cut_date = pd.to_datetime("2020-01-02")
train = feat[feat["date"] < cut_date].copy()
test  = feat[feat["date"] >= cut_date].copy()

X_train = train.drop(columns=["date","y"])
y_train = train["y"].astype(int)

X_test  = test.drop(columns=["date","y"])
y_test  = test["y"].astype(int)

# one-hot（pref_key）
X_train = pd.get_dummies(X_train, columns=["pref_key"], drop_first=False)
X_test  = pd.get_dummies(X_test, columns=["pref_key"], drop_first=False)

# 列揃え
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 追加項目
keep_cols = [c for c in X_train.columns if "30d" in c or "days_since" in c or "pref_key" in c]
X_train = X_train[keep_cols]
X_test = X_test[keep_cols]


pos_rate = y_train.mean()
scale_pos_weight = (1 - pos_rate) / max(pos_rate, 1e-9)

# =========================
# 6) 学習（LGBM）
# =========================
lgbm = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=63,
    min_child_samples=200,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    scale_pos_weight=scale_pos_weight,
)

lgbm.fit(X_train, y_train)

# =========================
# 7) 校正（sigmoid）=> キャリブレーションを行い、予測の信頼度を上げる
# =========================
cal = CalibratedClassifierCV(lgbm, method="sigmoid", cv=3)
cal.fit(X_train, y_train)

# =========================
# 8) 評価（必要最低限）
# =========================
p_raw = lgbm.predict_proba(X_test)[:,1]
p_cal = cal.predict_proba(X_test)[:,1]

print("AP(raw):", average_precision_score(y_test, p_raw))
print("Brier(raw):", brier_score_loss(y_test, p_raw)) #0に近いほど予測の精度が高い

print("AP(cal):", average_precision_score(y_test, p_cal))
print("Brier(cal):", brier_score_loss(y_test, p_cal))

# =========================
# 9) 保存（推論で必要なものだけ）
# =========================
artifact = {
    "models": {"raw": lgbm, "cal_sigmoid": cal},
    "feature_columns": X_train.columns.tolist(),
    "spec": {"horizon_days": 10, "threshold": None, "target": "I4p"},
    "cut_date": str(cut_date.date()),
}
joblib.dump(artifact, OUT_MODEL)
print(" 保存先:", OUT_MODEL)


[LightGBM] [Info] Number of positive: 18735, number of negative: 1715001
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.096587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 762
[LightGBM] [Info] Number of data points in the train set: 1733736, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.010806 -> initscore=-4.516776
[LightGBM] [Info] Start training from score -4.516776
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 12490, number of negative: 1143334
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.086182 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 728
[LightGBM] [Info] Num

### 30日間モデルの作成

In [8]:
import numpy as np
import pandas as pd
import joblib

from lightgbm import LGBMClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import average_precision_score, brier_score_loss

REFINED_CSV = "jma_shindo_refined.csv"
# OUT_MODEL   = "eq_pref_10d_I4p_model_ver30.joblib"
# OUT_MODEL   = "eq_pref_10d_I4p_model_ver30_90rm.joblib"
OUT_MODEL   = "eq_pref_10d_I4p_model_ver30_90rm_train.joblib"

# =========================
# 1) refined 読み込み
# =========================
df = pd.read_csv(REFINED_CSV, parse_dates=["ot"])
df = df.dropna(subset=["ot"]).copy()

df["date"] = df["ot"].dt.floor("D")
df["pref_key"] = df["pref_code"].astype(int)
df["maxI_num"] = pd.to_numeric(df["maxI_num"], errors="coerce")
df["mag"] = pd.to_numeric(df["mag"], errors="coerce")
df["dep_num"] = pd.to_numeric(df["dep_num"], errors="coerce")  # refinedで補完済みの想定

# =========================
# 2) 日次パネル作成（pref×date）
# =========================
daily_cnt = df.groupby(["pref_key","date"]).size().rename("cnt").reset_index()

df_i4 = df[df["maxI_num"].notna() & (df["maxI_num"] >= 4.0)]
daily_i4 = df_i4.groupby(["pref_key","date"]).size().rename("cnt_I4p").reset_index()

daily_mag = df.groupby(["pref_key","date"])["mag"].mean().rename("mag_mean").reset_index()
daily_dep = df.groupby(["pref_key","date"])["dep_num"].mean().rename("dep_mean").reset_index()

daily = (daily_cnt
         .merge(daily_i4, on=["pref_key","date"], how="left")
         .merge(daily_mag, on=["pref_key","date"], how="left")
         .merge(daily_dep, on=["pref_key","date"], how="left"))

daily["cnt_I4p"] = daily["cnt_I4p"].fillna(0).astype(int)
daily["mag_mean"] = daily["mag_mean"].fillna(0)   # ※magは補完しない方針だが、日次平均は0埋めして特徴量として扱う
daily["dep_mean"] = daily["dep_mean"].fillna(0)

# 全県×全日埋め（1..47固定）
all_dates = pd.date_range(daily["date"].min(), daily["date"].max(), freq="D")
prefs = np.arange(1, 48)
grid = pd.MultiIndex.from_product([prefs, all_dates], names=["pref_key","date"]).to_frame(index=False)

daily_full = grid.merge(daily, on=["pref_key","date"], how="left")
daily_full["cnt"] = daily_full["cnt"].fillna(0).astype(int)
daily_full["cnt_I4p"] = daily_full["cnt_I4p"].fillna(0).astype(int)
daily_full["mag_mean"] = daily_full["mag_mean"].fillna(0)
daily_full["dep_mean"] = daily_full["dep_mean"].fillna(0)
daily_full = daily_full.sort_values(["pref_key","date"]).reset_index(drop=True)

# =========================
# 3) 特徴量
# =========================
def add_features(d):
    d = d.copy()
    g = d.groupby("pref_key", group_keys=False)

    for w in [3,7,14]:
        d[f"cnt_{w}d"]      = g["cnt"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"cntI4_{w}d"]    = g["cnt_I4p"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"mag_{w}d_mean"] = g["mag_mean"].rolling(w, min_periods=1).mean().reset_index(level=0, drop=True)
        d[f"dep_{w}d_mean"] = g["dep_mean"].rolling(w, min_periods=1).mean().reset_index(level=0, drop=True)

    # for w in [10,30,90]:
    for w in [10,30]:
        d[f"cnt_{w}d"]   = g["cnt"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"cntI4_{w}d"] = g["cnt_I4p"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)

    # days_since_any_eq
    d["any_eq_today"] = (d["cnt"] > 0).astype(int)
    last_eq = np.where(d["any_eq_today"].to_numpy() > 0, d["date"].to_numpy(dtype="datetime64[D]"), np.datetime64("NaT"))
    d["last_eq_date"] = pd.Series(last_eq)
    d["last_eq_date"] = g["last_eq_date"].ffill()
    diff = (d["date"].to_numpy(dtype="datetime64[D]") - d["last_eq_date"].to_numpy(dtype="datetime64[D]")).astype("timedelta64[D]")
    d["days_since_any_eq"] = np.where(pd.isna(d["last_eq_date"]), 9999, diff.astype(int))

    # days_since_I4p
    d["i4_today"] = (d["cnt_I4p"] > 0).astype(int)
    last_i4 = np.where(d["i4_today"].to_numpy() > 0, d["date"].to_numpy(dtype="datetime64[D]"), np.datetime64("NaT"))
    d["last_i4_date"] = pd.Series(last_i4)
    d["last_i4_date"] = g["last_i4_date"].ffill()
    diff2 = (d["date"].to_numpy(dtype="datetime64[D]") - d["last_i4_date"].to_numpy(dtype="datetime64[D]")).astype("timedelta64[D]")
    d["days_since_I4p"] = np.where(pd.isna(d["last_i4_date"]), 9999, diff2.astype(int))

    d = d.drop(columns=["last_eq_date","last_i4_date"], errors="ignore")
    return d

feat = add_features(daily_full)

# =========================
# 4) 目的変数 y（次の30日でI4+が1回でもあれば1）
#    ※未来リーク防止：prefごとにshift
# =========================
H = 30
g = feat.groupby("pref_key", group_keys=False)
future_i4 = g["cnt_I4p"].rolling(H, min_periods=1).sum().reset_index(level=0, drop=True).shift(-H)
feat["y"] = (future_i4.fillna(0) > 0).astype(int)

# =========================
# 5) 学習/テスト（時系列ホールドアウト）
# =========================
cut_date = pd.to_datetime("2020-01-02")
train = feat[feat["date"] < cut_date].copy()
test  = feat[feat["date"] >= cut_date].copy()

X_train = train.drop(columns=["date","y"])
y_train = train["y"].astype(int)

X_test  = test.drop(columns=["date","y"])
y_test  = test["y"].astype(int)

# one-hot（pref_key）
X_train = pd.get_dummies(X_train, columns=["pref_key"], drop_first=False)
X_test  = pd.get_dummies(X_test, columns=["pref_key"], drop_first=False)

# 列揃え
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# 追加項目
keep_cols = [c for c in X_train.columns if "30d" in c or "days_since" in c or "pref_key" in c]
X_train = X_train[keep_cols]
X_test = X_test[keep_cols]

pos_rate = y_train.mean()
scale_pos_weight = (1 - pos_rate) / max(pos_rate, 1e-9)

# =========================
# 6) 学習（LGBM）
# =========================
lgbm = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=63,
    min_child_samples=200,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    scale_pos_weight=scale_pos_weight,
)

lgbm.fit(X_train, y_train)

# =========================
# 7) 校正（sigmoid）
# =========================
cal = CalibratedClassifierCV(lgbm, method="sigmoid", cv=3)
cal.fit(X_train, y_train)

# =========================
# 8) 評価
# =========================
p_raw = lgbm.predict_proba(X_test)[:,1]
p_cal = cal.predict_proba(X_test)[:,1]

print("AP(raw):", average_precision_score(y_test, p_raw))
print("Brier(raw):", brier_score_loss(y_test, p_raw))
print("AP(cal):", average_precision_score(y_test, p_cal))
print("Brier(cal):", brier_score_loss(y_test, p_cal))

# =========================
# 9) 保存
# =========================
artifact = {
    "models": {"raw": lgbm, "cal_sigmoid": cal},
    "feature_columns": X_train.columns.tolist(),
    "spec": {"horizon_days": 30, "threshold": None, "target": "I4p"},
    "cut_date": str(cut_date.date()),
}
joblib.dump(artifact, OUT_MODEL)
print("saved:", OUT_MODEL)

[LightGBM] [Info] Number of positive: 50413, number of negative: 1683323
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.098002 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 762
[LightGBM] [Info] Number of data points in the train set: 1733736, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.029078 -> initscore=-3.508276
[LightGBM] [Info] Start training from score -3.508276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 33609, number of negative: 1122215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035611 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 735
[LightGBM] [Info] Num

In [9]:
import numpy as np
import pandas as pd
import joblib
from functools import lru_cache

# =========================
# 設定
# =========================
# ARTIFACT_10 = "eq_pref_10d_I4p_model_ver10.joblib"
# ARTIFACT_30 = "eq_pref_10d_I4p_model_ver30.joblib"
# ARTIFACT_10_90rm = "eq_pref_10d_I4p_model_ver10_90rm.joblib"
# ARTIFACT_30_90rm = "eq_pref_10d_I4p_model_ver30_90rm.joblib"
ARTIFACT_10_90rm = "eq_pref_10d_I4p_model_ver10_90rm_train.joblib"
ARTIFACT_30_90rm = "eq_pref_10d_I4p_model_ver30_90rm_train.joblib"
REFINED_CSV = "jma_shindo_refined.csv"

# =========================
# 0) モデル読み込み
# =========================
# a10 = joblib.load(ARTIFACT_10)
# a30 = joblib.load(ARTIFACT_30)
a10 = joblib.load(ARTIFACT_10_90rm)
a30 = joblib.load(ARTIFACT_30_90rm)
model10 = a10["models"]["cal_sigmoid"]
model30 = a30["models"]["cal_sigmoid"]
feature_columns = a10["feature_columns"]

# =========================
# 1) 最終データ日 & 都道府県マップ
# =========================
def get_last_date_and_pref_map(refined_csv: str):
    df = pd.read_csv(refined_csv, usecols=["ot", "pref_code", "edit_name"])
    df["ot"] = pd.to_datetime(df["ot"], errors="coerce")
    df = df.dropna(subset=["ot"]).copy()
    last_date = df["ot"].max().floor("D")

    df["pref_code"] = pd.to_numeric(df["pref_code"], errors="coerce")
    df = df.dropna(subset=["pref_code","edit_name"]).copy()
    df["pref_code"] = df["pref_code"].astype(int)
    pref_map = df.groupby("pref_code")["edit_name"].agg(lambda s: s.value_counts().index[0]).to_dict()
    return last_date, pref_map

LAST_DATE, pref_key_to_name = get_last_date_and_pref_map(REFINED_CSV)

# =========================
# 2) 日次作成（指定日まで）
# =========================
@lru_cache(maxsize=16)
def build_daily_full_upto_cached(upto_date_str: str) -> pd.DataFrame:
    upto = pd.to_datetime(upto_date_str).floor("D")

    df = pd.read_csv(REFINED_CSV)
    df["ot"] = pd.to_datetime(df["ot"], errors="coerce")
    df = df.dropna(subset=["ot"]).copy()
    df["date"] = df["ot"].dt.floor("D")
    df = df[df["date"] <= upto].copy()

    df["pref_key"] = pd.to_numeric(df["pref_code"], errors="coerce").astype(int)
    df["maxI_num"] = pd.to_numeric(df["maxI_num"], errors="coerce")
    df["mag_num"] = pd.to_numeric(df["mag"], errors="coerce")
    df["dep_num"] = pd.to_numeric(df["dep_num"], errors="coerce")

    # dep_numは中央値補完（方針）
    dep_median = df["dep_num"].median()
    df["dep_num"] = df["dep_num"].fillna(dep_median)

    # 日次集約
    daily_cnt = df.groupby(["pref_key","date"]).size().rename("cnt").reset_index()

    df_i4 = df[df["maxI_num"].notna() & (df["maxI_num"] >= 4.0)]
    daily_i4 = df_i4.groupby(["pref_key","date"]).size().rename("cnt_I4p").reset_index()

    daily_maxI = df.groupby(["pref_key","date"])["maxI_num"].max().rename("maxI_max").reset_index()
    daily_mag_mean = df.groupby(["pref_key","date"])["mag_num"].mean().rename("mag_mean").reset_index()
    daily_dep_mean = df.groupby(["pref_key","date"])["dep_num"].mean().rename("dep_mean").reset_index()

    daily = (daily_cnt
             .merge(daily_i4, on=["pref_key","date"], how="left")
             .merge(daily_maxI, on=["pref_key","date"], how="left")
             .merge(daily_mag_mean, on=["pref_key","date"], how="left")
             .merge(daily_dep_mean, on=["pref_key","date"], how="left"))

    daily["cnt"] = daily["cnt"].fillna(0).astype(int)
    daily["cnt_I4p"] = daily["cnt_I4p"].fillna(0).astype(int)
    daily["maxI_max"] = daily["maxI_max"].fillna(0.0)
    daily["mag_mean"] = daily["mag_mean"].fillna(0.0)
    daily["dep_mean"] = daily["dep_mean"].fillna(0.0)

    # pref×全日埋め（1..47固定）
    all_dates = pd.date_range(daily["date"].min(), upto, freq="D")
    prefs = np.arange(1, 48)
    grid = pd.MultiIndex.from_product([prefs, all_dates], names=["pref_key","date"]).to_frame(index=False)

    daily_full = grid.merge(daily, on=["pref_key","date"], how="left")
    daily_full["cnt"] = daily_full["cnt"].fillna(0).astype(int)
    daily_full["cnt_I4p"] = daily_full["cnt_I4p"].fillna(0).astype(int)
    daily_full["maxI_max"] = daily_full["maxI_max"].fillna(0.0)
    daily_full["mag_mean"] = daily_full["mag_mean"].fillna(0.0)
    daily_full["dep_mean"] = daily_full["dep_mean"].fillna(0.0)

    return daily_full.sort_values(["pref_key","date"]).reset_index(drop=True)

# =========================
# 3) 特徴量
# =========================
def add_features(daily_full: pd.DataFrame) -> pd.DataFrame:
    d = daily_full.copy()
    g = d.groupby("pref_key", group_keys=False)

    for w in [3,7,14]:
        d[f"cnt_{w}d"] = g["cnt"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"cntI4_{w}d"] = g["cnt_I4p"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"maxI_{w}d_max"] = g["maxI_max"].rolling(w, min_periods=1).max().reset_index(level=0, drop=True)
        d[f"mag_{w}d_mean"] = g["mag_mean"].rolling(w, min_periods=1).mean().reset_index(level=0, drop=True)
        d[f"dep_{w}d_mean"] = g["dep_mean"].rolling(w, min_periods=1).mean().reset_index(level=0, drop=True)

    for w in [10,30,90]:
        d[f"cnt_{w}d"] = g["cnt"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)
        d[f"cntI4_{w}d"] = g["cnt_I4p"].rolling(w, min_periods=1).sum().reset_index(level=0, drop=True)

    # days_since_any_eq
    d["any_eq_today"] = (d["cnt"] > 0).astype(int)
    last_eq = np.where(d["any_eq_today"].to_numpy() > 0,
                       d["date"].to_numpy(dtype="datetime64[D]"),
                       np.datetime64("NaT"))
    d["last_eq_date"] = pd.Series(last_eq)
    d["last_eq_date"] = g["last_eq_date"].ffill()
    diff = (d["date"].to_numpy(dtype="datetime64[D]") - d["last_eq_date"].to_numpy(dtype="datetime64[D]")).astype("timedelta64[D]")
    d["days_since_any_eq"] = np.where(pd.isna(d["last_eq_date"]), 9999, diff.astype(int))

    # days_since_I4p
    d["i4_today"] = (d["cnt_I4p"] > 0).astype(int)
    last_i4 = np.where(d["i4_today"].to_numpy() > 0,
                       d["date"].to_numpy(dtype="datetime64[D]"),
                       np.datetime64("NaT"))
    d["last_i4_date"] = pd.Series(last_i4)
    d["last_i4_date"] = g["last_i4_date"].ffill()
    diff2 = (d["date"].to_numpy(dtype="datetime64[D]") - d["last_i4_date"].to_numpy(dtype="datetime64[D]")).astype("timedelta64[D]")
    d["days_since_I4p"] = np.where(pd.isna(d["last_i4_date"]), 9999, diff2.astype(int))

    d["cnt_14d_delta"] = g["cnt_14d"].diff().fillna(0)
    d = d.drop(columns=["last_eq_date","last_i4_date"], errors="ignore")
    return d

# =========================
# 4) ベースライン（履歴から推定）
# =========================
def compute_baseline_from_history(feat: pd.DataFrame, H: int, upto_date: pd.Timestamp):
    d = feat[feat["date"] <= upto_date].copy()
    g = d.groupby("pref_key", group_keys=False)
    future_i4 = g["cnt_I4p"].rolling(H, min_periods=1).sum().reset_index(level=0, drop=True).shift(-H)
    d["y_tmp"] = (future_i4.fillna(0) > 0).astype(int)
    d = d.dropna(subset=["y_tmp"]).copy()

    base_all = float(d["y_tmp"].mean()) if len(d) else 0.0
    base_by_pref = d.groupby("pref_key")["y_tmp"].mean().to_dict() if len(d) else {}
    return base_all, base_by_pref

# =========================
# 5) 予測（10日＋30日）
# =========================
def predict_pref_10_30(base_date: str, pref_code: int) -> dict:
    base_date = pd.to_datetime(base_date).floor("D")
    pref_code = int(pref_code)

    # 予測範囲チェック（最終データ日より未来は不可）
    if base_date > LAST_DATE:
        raise ValueError(f"base_date={base_date.date()} は最終データ日={LAST_DATE.date()}より未来です。")

    daily_full = build_daily_full_upto_cached(str(base_date.date()))
    feat = add_features(daily_full)

    day_all = feat.loc[feat["date"] == base_date].copy()
    if day_all.empty:
        raise ValueError("base_date がデータ範囲外です。")

    X_day = day_all.drop(columns=["date"])
    X_day = pd.get_dummies(X_day, columns=["pref_key"], drop_first=False)
    X_day = X_day.reindex(columns=feature_columns, fill_value=0)

    p10_all = model10.predict_proba(X_day)[:, 1]
    p30_all = model30.predict_proba(X_day)[:, 1]

    day_all["p10"] = p10_all
    day_all["p30"] = p30_all

    r = day_all.loc[day_all["pref_key"] == pref_code].iloc[0]

    base10_all, base10_by = compute_baseline_from_history(feat, H=10, upto_date=base_date)
    base30_all, base30_by = compute_baseline_from_history(feat, H=30, upto_date=base_date)

    base10_pref = float(base10_by.get(pref_code, np.nan))
    base30_pref = float(base30_by.get(pref_code, np.nan))

    # 数値（％）は最後に2桁に丸める
    p10 = float(r["p10"]) * 100
    p30 = float(r["p30"]) * 100
    b10_all = base10_all * 100
    b30_all = base30_all * 100
    b10_pref = base10_pref * 100 if not np.isnan(base10_pref) else None
    b30_pref = base30_pref * 100 if not np.isnan(base30_pref) else None

    result = {
        # "最終データ日（予測可能な最大日）": str(LAST_DATE.date()),
        "基準日": str(base_date.date()),
        "都道府県コード": pref_code,
        "都道府県名": pref_key_to_name.get(pref_code, "不明"),

        "今後10日以内に震度4以上が発生する確率（％）": p10,
        "今後30日以内に震度4以上が発生する確率（％）": p30,

        "全国平均発生確率_10日（％）": b10_all,
        "全国平均発生確率_30日（％）": b30_all,

        "当該都道府県の過去平均発生確率_10日（％）": b10_pref,
        "当該都道府県の過去平均発生確率_30日（％）": b30_pref,

        "全国平均との差_10日（ポイント）": p10 - b10_all,
        "全国平均との差_30日（ポイント）": p30 - b30_all,
    }
    if b10_pref is not None:
        result["過去平均との差_10日（ポイント）"] = p10 - b10_pref
    if b30_pref is not None:
        result["過去平均との差_30日（ポイント）"] = p30 - b30_pref

    # ★ 表示用に2桁へ統一（Noneはそのまま）
    for k, v in list(result.items()):
        if isinstance(v, (float, np.floating)):
            result[k] = round(float(v), 2)

    return result

# =========================
# 6) 縦長 + 色付け（全国平均との差でHigh/Low）
# =========================
def result_to_vertical_df(result: dict) -> pd.DataFrame:
    return pd.DataFrame([result]).T.reset_index().rename(columns={"index":"項目", 0:"値"})

def style_risk_vertical(df_long: pd.DataFrame,
                        high_pt=2.0, low_pt=-1.0):
    """
    判定は「全国平均との差（ポイント）」で行う
    - 高い：差が +high_pt 以上 → 赤
    - 低い：差が  low_pt 以下 → 水色
    - それ以外：無色
    """
    def get_float(item):
        s = df_long.loc[df_long["項目"] == item, "値"]
        if len(s) == 0:
            return None
        try:
            return float(s.iloc[0])
        except:
            return None

    d10 = get_float("全国平均との差_10日（ポイント）")
    d30 = get_float("全国平均との差_30日（ポイント）")

    def judge(d):
        if d is None:
            return "none"
        if d >= high_pt:
            return "high"
        if d <= low_pt:
            return "low"
        return "none"

    j10 = judge(d10)
    j30 = judge(d30)

    def row_style(row):
        item = row["項目"]
        if item == "今後10日以内に震度4以上が発生する確率（％）":
            if j10 == "high":
                return ["text-align:left", "background-color:#ffcccc"]
            if j10 == "low":
                return ["text-align:left", "background-color:#cceeff"]
        if item == "今後30日以内に震度4以上が発生する確率（％）":
            if j30 == "high":
                return ["text-align:left", "background-color:#ffcccc"]
            if j30 == "low":
                return ["text-align:left", "background-color:#cceeff"]
        return ["text-align:left", ""]

    return (df_long.style
            .apply(row_style, axis=1)
            .set_properties(subset=["項目"], **{"text-align":"left"})
            .set_properties(subset=["値"], **{"text-align":"left"})
           )

### matplotlibの日本語ラベル表示用

In [10]:
!pip install -q japanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### 予測モデルの実行

In [11]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import japanize_matplotlib

# =========================
# 1) 判定ロジック（平均±σ）
# =========================
ymax = 15.0 # 1919-2025までの最大確率に調整済
def classify_level_by_sigma(x: float, mean: float, std: float, k: float = 1.0) -> str:
    hi = mean + k * std
    lo = mean - k * std
    if x >= hi:
        return "高い"
    if x <= lo:
        return "低い"
    return "どちらでもない"

def build_thresholds_from_probs(p10_percent: np.ndarray, p30_percent: np.ndarray):
    def stats(arr):
        arr = np.asarray(arr)
        return {
            "mean": float(arr.mean()),
            "std": float(arr.std(ddof=0)),
            "p05": float(np.percentile(arr, 5)),
            "p95": float(np.percentile(arr, 95)),
        }
    return {"p10": stats(p10_percent), "p30": stats(p30_percent)}

def add_level_labels(result: dict, TH: dict, k: float = 1.0) -> dict:
    p10 = float(result["今後10日以内に震度4以上が発生する確率（％）"])
    p30 = float(result["今後30日以内に震度4以上が発生する確率（％）"])
    result["判定_10日（平均±σ）"] = classify_level_by_sigma(p10, TH["p10"]["mean"], TH["p10"]["std"], k=k)
    result["判定_30日（平均±σ）"] = classify_level_by_sigma(p30, TH["p30"]["mean"], TH["p30"]["std"], k=k)
    return result

def style_risk_vertical_by_level(df_long: pd.DataFrame) -> "pd.io.formats.style.Styler":
    def row_style(row):
        item = row["項目"]
        if item not in ["今後10日以内に震度4以上が発生する確率（％）",
                        "今後30日以内に震度4以上が発生する確率（％）"]:
            return ["text-align:left", ""]

        judge_key = "判定_10日（平均±σ）" if "10日" in item else "判定_30日（平均±σ）"
        judge = df_long.loc[df_long["項目"] == judge_key, "値"]
        judge = judge.iloc[0] if len(judge) else ""

        if judge == "高い":
            return ["text-align:left", "background-color:#ffcccc"]
        if judge == "低い":
            return ["text-align:left", "background-color:#cceeff"]
        return ["text-align:left", ""]

    return (
        df_long.style
        .apply(row_style, axis=1)
        .set_properties(subset=["項目"], **{"text-align": "left"})
        # .set_properties(subset=["値"], **{"text-align": "left"})
    )

# =========================
# 2) グラフ
# =========================
def plot_probs(result: dict, ymax: float = 11.0):
    p10 = float(result["今後10日以内に震度4以上が発生する確率（％）"])
    p30 = float(result["今後30日以内に震度4以上が発生する確率（％）"])
    b10 = float(result["全国平均発生確率_10日（％）"])
    b30 = float(result["全国平均発生確率_30日（％）"])

    x = [10, 30]
    plt.figure()
    plt.plot(x, [p10, p30], marker="o", label="指定都道府県（予測確率）")
    plt.plot(x, [b10, b30], marker="o", label="全国平均（参考）")
    plt.xticks([10, 30])
    plt.xlabel("基準日からの経過日数（日）")
    plt.ylabel("発生確率（％）")
    plt.title("震度4以上の発生確率（10日/30日）")
    plt.ylim(0, ymax)
    plt.grid(True, axis="y", alpha=0.3)
    plt.legend()
    plt.show()

# =========================
# 3) UI本体（まとめ）
# =========================
def build_eq_ui(
    pref_key_to_name: dict,
    LAST_DATE: str,
    predict_fn,                 # predict_pref_10_30
    to_vertical_df_fn,          # result_to_vertical_df
    style_by_pt_fn=None,        # style_risk_vertical（全国平均との差で色付けするやつ）
    TH: dict = None,            # 平均±σの閾値辞書
    ymax: float = 11.0,
):
    # options: (表示, value)
    pref_options = [(f"{code:02d} {name}", int(code)) for code, name in sorted(pref_key_to_name.items())]

    w_pref = widgets.Dropdown(options=pref_options, value=pref_options[0][1],
                              description="都道府県:", layout=widgets.Layout(width="320px"))
    w_date = widgets.DatePicker(description="基準日:",
                                value=pd.Timestamp(LAST_DATE).to_pydatetime().date(),
                                layout=widgets.Layout(width="320px"))

    # “色付け方式” を選べるように（pt方式 or 平均±σ）
    w_mode = widgets.ToggleButtons(
        options=[("全国平均との差(pt)", "pt"), ("平均±σ", "sigma")],
        value="sigma" if TH is not None else "pt",
        description="判定:",
        layout=widgets.Layout(width="420px")
    )

    # pt方式用（既存の style_risk_vertical がある場合のみ使う）
    w_high = widgets.FloatText(value=2.0, description="高い(+pt):", layout=widgets.Layout(width="220px"))
    w_low = widgets.FloatText(value=-1.0, description="低い(-pt):", layout=widgets.Layout(width="220px"))

    w_k = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1,
                              description="σ係数k:", layout=widgets.Layout(width="320px"))

    btn = widgets.Button(description="予測する", button_style="primary")
    out = widgets.Output()

    def show_table(result: dict):
        df_long = to_vertical_df_fn(result)

        if w_mode.value == "pt":
            if style_by_pt_fn is None:
                display(df_long)  # fallback
            else:
                display(style_by_pt_fn(df_long, high_pt=float(w_high.value), low_pt=float(w_low.value)))
        else:
            if TH is None:
                display(df_long)  # fallback
            else:
                result2 = add_level_labels(result.copy(), TH, k=float(w_k.value))
                df_long2 = to_vertical_df_fn(result2)
                display(style_risk_vertical_by_level(df_long2))

    def on_click(_):
        with out:
            clear_output(wait=True)
            if w_date.value is None:
                print("基準日が未入力です。")
                return

            try:
                result = predict_fn(str(w_date.value), int(w_pref.value))
                show_table(result)
                plot_probs(result, ymax=ymax)
            except Exception as e:
                print("エラーが出ました：")
                print(e)

    btn.on_click(on_click)

    # UIレイアウト
    row1 = widgets.HBox([w_pref, w_date])
    row2 = widgets.HBox([w_mode, btn])
    row3 = widgets.HBox([w_high, w_low])
    row4 = widgets.HBox([w_k])

    display(row1, row2, row3, row4, out)

    return {
        "pref": w_pref, "date": w_date, "mode": w_mode,
        "high_pt": w_high, "low_pt": w_low, "k": w_k,
        "button": btn, "output": out
    }

# =========================
# 4) 実行
# =========================
ui = build_eq_ui(
    pref_key_to_name=pref_key_to_name,
    LAST_DATE=LAST_DATE,
    predict_fn=predict_pref_10_30,
    to_vertical_df_fn=result_to_vertical_df,
    style_by_pt_fn=style_risk_vertical,   # 既存がある前提
    # TH=TH,
    ymax=ymax
)


Output()